In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../input/first-gop-debate-twitter-sentiment/Sentiment.csv')
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [3]:
data = data[data.sentiment != "Neutral"]
data['sentiment']= pd.get_dummies(data['sentiment'], drop_first = True)
data = data.reset_index(drop=True)
data.head()

,text,sentiment
0,RT @ScottWalker: Didn't catch the full #GOPdeb...,1
1,RT @RobGeorge: That Carly Fiorina is trending ...,1
2,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,1
3,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",1
4,RT @warriorwoman91: I liked her and was happy ...,0


In [4]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [5]:
corpus = []

for i in range(0, data.shape[0]):
    tweet = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    tweet = tweet.lower()
    tweet = word_tokenize(tweet)
    # Reduce words to their root form
    tweet = [WordNetLemmatizer().lemmatize(w) for w in tweet if not w in set(stopwords.words('english'))]
    # Lemmatize verbs by specifying pos
    tweet = [WordNetLemmatizer().lemmatize(w, pos='v') for w in tweet if not w in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

print(corpus[0:3])

['rt scottwalker catch full gopdebate last night scott best line second walker http co zsff', 'rt robgeorge carly fiorina trend hour debate men complete gopdebate say', 'rt danscavino gopdebate w realdonaldtrump deliver highest rat history presidential debate trump http co']


##applying TF-IDF to check the most relevant words after preprocessing , so we can remove some of these word to improve the model accuracy

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(corpus)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
pd.set_option('display.max_rows', None)
df.head(20)


,TF-IDF
zsff,0.453518
catch,0.344600
full,0.330405
second,0.313011
line,0.299403
scottwalker,0.282078
scott,0.274517
best,0.249607
walker,0.235920
last,0.182394


In [7]:
for i in range(0, len(corpus)):
    corpus[i] = re.sub('co','', corpus[i])
    corpus[i] = re.sub('rt','', corpus[i])
    corpus[i] = re.sub('http','', corpus[i])
    

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(corpus)
encoded_docs = tokenizer.texts_to_sequences(corpus)
padded_sequence = pad_sequences(encoded_docs,maxlen=200)


In [9]:
print(tokenizer.word_index['trump'])

4


In [10]:
print(corpus[0])
print(encoded_docs[0])

 sttwalker catch full gopdebate last night stt best line send walker   zsff
[252, 779, 564, 1, 15, 13, 204, 114, 354, 224, 72, 5278]


In [11]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  252  779  564    1   15   13  204  114
  354 

In [12]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant

vocab_size = len(tokenizer.word_index) + 1
embedding_vector_length = 200


model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,
                    input_length=25) )
model.add(SpatialDropout1D(0.2))
model.add(LSTM(3, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 200)           2598000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 25, 200)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 3)                 2448      
_________________________________________________________________
dropout (Dropout)            (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 4         
Total params: 2,600,452
Trainable params: 2,600,452
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# converting the targets to numpy array to feed it into the model
target = np.asarray(data['sentiment'])

In [14]:
MODEL = model.fit(padded_sequence,target,validation_split=0.2, epochs=5, batch_size=256)

Epoch 1/5
34/34 [==============================] - 25s 635ms/step - loss: 0.6584 - accuracy: 0.6976 - val_loss: 0.5307 - val_accuracy: 0.7968
Epoch 2/5
34/34 [==============================] - 20s 599ms/step - loss: 0.5157 - accuracy: 0.7948 - val_loss: 0.4866 - val_accuracy: 0.7968
Epoch 3/5
34/34 [==============================] - 20s 592ms/step - loss: 0.4550 - accuracy: 0.8020 - val_loss: 0.4494 - val_accuracy: 0.8010
Epoch 4/5
34/34 [==============================] - 20s 602ms/step - loss: 0.3967 - accuracy: 0.8399 - val_loss: 0.3983 - val_accuracy: 0.8257
Epoch 5/5
34/34 [==============================] - 21s 607ms/step - loss: 0.3282 - accuracy: 0.8872 - val_loss: 0.3480 - val_accuracy: 0.8518


In [15]:
test_word ='love trump'
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
print(prediction)

1


finally this is the best accuracy i could get after hyperparametes tuning + TF-IDF as guidance for removing some words